# Section 5.5 – Travaux pratiques

Ce notebook illustre l’application de **tests d’homogénéité** pour comparer des caractéristiques entre deux groupes. Il s’agit de tests sur des moyennes ou des proportions lorsque les échantillons sont indépendants ou appariés, conformément au chapitre 5.5 du polycopié.


## 5.5.1 Échantillons indépendants

Lorsque deux échantillons proviennent de populations différentes et que les observations sont indépendantes, on compare les paramètres à l’aide de tests adaptés selon que les variances sont connues ou non.


### Exemple 54 – Comparaison de deux machines (variances connues)

On dispose des poids (en grammes) de paquets de cacao remplis par deux machines. Les variances des poids sont supposées connues, respectivement $(1{,}3)^2$ et $(0{,}9)^2$. On souhaite tester si les moyennes sont différentes.


In [ ]:

# Poids des paquets pour les deux machines
machine1 <- c(106.70, 107.02, 107.15, 107.22, 107.41, 106.39, 107.47, 107.61, 107.38, 107.22)
machine2 <- c(107.68, 106.69, 107.24, 107.69, 106.97, 107.52, 106.22, 107.23, 107.32)

# Moyennes observées
mean1 <- mean(machine1)
mean2 <- mean(machine2)

# Variances théoriques connues
sigma1_sq <- (1.3)^2
sigma2_sq <- (0.9)^2

# Statistique de test pour comparaison de moyennes avec variances connues
z_obs <- (mean1 - mean2) / sqrt(sigma1_sq/length(machine1) + sigma2_sq/length(machine2))

# P‑valeur bilatérale
p_value <- 2 * (1 - pnorm(abs(z_obs)))

c(z_obs = z_obs, p_value = p_value)


Si la p‑valeur est inférieure au niveau de risque (par exemple 5 %), on rejette l’hypothèse $H_0 : \mu_1 = \mu_2$ et on conclut que les machines sont réglées différemment.


### Exemple 55 – Deux producteurs (variances inconnues)

Les teneurs en viande (en grammes) de boîtes provenant de deux producteurs différents sont données ci‑dessous. On suppose les données normalement distribuées. On veut tester si les moyennes diffèrent.


In [ ]:

# Teneur en viande pour les deux producteurs
prod1 <- c(2.12, 12.03, 13.58, 13.38, 11.81, 15.92, 13.65)
prod2 <- c(14.81, 13.93, 14.91, 15.87, 15.62, 15.39)

# Test t pour deux échantillons indépendants (variances non supposées égales)
res_ttest <- t.test(prod1, prod2)

# Moyennes observées et p‑valeur
a <- mean(prod1)
b <- mean(prod2)
p_val <- res_ttest$p.value
c(mean_prod1 = a, mean_prod2 = b, p_value = p_val)


Par défaut, $t.test()$ utilise la version sans hypothèse d’égalité des variances (test de Welch). Si l’on souhaite supposer des variances égales, on ajoute l’argument $var.equal = TRUE$.


### Exemple 56 – Comparaison de deux lots de tasses (variances égales)

On dispose de deux échantillons de résistances (en MPa) mesurées sur des tasses issues de deux lots. On souhaite tester si les résistances moyennes sont différentes en supposant des variances égales.


In [ ]:

# Résistance des tasses pour les deux lots
lot1 <- c(31.70, 31.98, 32.24, 32.35, 31.18, 32.19, 32.63, 31.19, 31.54, 31.89)
lot2 <- c(31.61, 31.10, 31.20, 31.11, 32.66, 31.15, 31.71, 31.22, 31.16, 31.21)

# Test t avec égalité des variances
res_ttest_equal <- t.test(lot1, lot2, var.equal = TRUE)

# P‑valeur
res_ttest_equal$p.value


Si la p‑valeur est faible, on conclut à une différence significative entre les lots.


### Exemple 57 – Comparaison de proportions dans deux périodes

Un producteur veut savoir si une nouvelle présentation a augmenté sa part de marché. On compare deux échantillons :

- Avant la nouvelle présentation : 54 succès sur 230 observations.
- Après la nouvelle présentation : 110 succès sur 340 observations.

On teste si la proportion après est supérieure à la proportion avant.


In [ ]:

# Nombre de succès et tailles d'échantillon
x <- c(54, 110)
n <- c(230, 340)

# Test de proportions (bilatéral par défaut)
res_prop_test <- prop.test(x = x, n = n)

# P‑valeur bilatérale
res_prop_test$p.value

# P‑valeur unilatérale (H1: p2 > p1)
res_prop_test_one_sided <- prop.test(x = x, n = n, alternative = "less") # p1 < p2 revient à p2 > p1
res_prop_test_one_sided$p.value


Si la p‑valeur unilatérale est inférieure à 5 %, on conclut que la nouvelle présentation a significativement augmenté la part de marché.


## 5.5.2 Échantillons appariés

Lorsque les mêmes individus sont mesurés deux fois (par exemple avant et après traitement), on utilise un **paired t-test** pour comparer les moyennes des différences.


### Exemple 58 – Variation de poids avant et après traitement

On examine le poids de cinq malades avant et après un traitement et on veut tester si le traitement augmente le poids en moyenne.


In [ ]:

# Poids avant et après traitement
after <- c(83.76, 64.13, 101.81, 56.63, 68.21)
before <- c(80.82, 60.12, 102.52, 51.65, 65.96)

# Test t apparié (H0: pas de différence, H1: augmentation du poids)
res_paired <- t.test(after, before, paired = TRUE, alternative = "greater")

res_paired$p.value


Une p‑valeur inférieure à 0,05 indique que le traitement augmente significativement le poids des patients.

### Exemple 59 – Test de Mac Nemar

Pour comparer deux traitements sur des données binaires appariées (succès/échec), on utilise le test de Mac Nemar. Le jeu de données se résume par une table de contingence 2×2 comptant les combinaisons ($n01$, $n10$, etc.).


In [ ]:

# Exemple : données binaires pour deux traitements sur les mêmes individus
# n01 : nombre de fois où le premier traitement = 0 et le second = 1, etc.
table_data <- matrix(c(30, 5, 3, 12), nrow = 2,
                     dimnames = list(Traitement1 = c("Succès", "Échec"),
                                     Traitement2 = c("Succès", "Échec")))

# Afficher la table
addmargins(table_data)

# Test de Mac Nemar
res_mcnemar <- mcnemar.test(table_data, correct = TRUE)

res_mcnemar$p.value


Le test de Mac Nemar est utilisé pour déterminer si les proportions de succès diffèrent entre deux traitements pour les mêmes sujets. Une p‑valeur faible conduit à rejeter l’hypothèse d’égalité des proportions.

## Conclusion

Cette section montre comment utiliser R pour comparer deux groupes, qu’il s’agisse d’échantillons indépendants ou appariés. Les fonctions $t.test()$, $prop.test()$ et $mcnemar.test()$ permettent de réaliser rapidement ces tests en renvoyant les statistiques utiles.
